## Statistics in Presidential Debates

In this notebook, I scrape data from the Presidential Debates from the Commission of Presidential Debates website: https://www.debates.org/voter-education/debate-transcripts/

I analyze the counts of specific values from each of the speeches and sentiment score each of the candidate's sentiment score using the VADAR NLP Sentiment analysis toolkit.

Using `requests` and `BeautifulSoup` to find all links/ URLs on the website and use the links found to get the text from each presidential debate.

This project was inspired by a project I did for my DataX class.



In [1]:
import requests
import nltk
from textblob import TextBlob
import numpy as np
import bs4 as bs
from collections import Counter
import re
import pandas as pd
source = requests.get("https://www.debates.org/voter-education/debate-transcripts/") 
soup = bs.BeautifulSoup(source.content, features='html.parser')

In [2]:
urllist = []
titlelist = []
for a in soup.find_all('a'):
        stringurl = "https://www.debates.org" + a.get('href')
        if stringurl not in urllist and 'debate-transcript' in a.get('href'):
            urllist.append(stringurl)
            titlelist.append(a.text)

titlelist

#deal with first val/last val
#will have to combine
#print(titlelist)

['Debate Transcripts',
 'October 19, 2016 Debate Transcript',
 'October 9, 2016 Debate Transcript',
 'October 4, 2016 Debate Transcript',
 'September 26, 2016 Debate Transcript',
 'October 22, 2012 Debate Transcript',
 'October 16, 2012 Debate Transcript',
 'October 11, 2012 Debate Transcript',
 'October 3, 2012 Debate Transcript',
 'September 26, 2008 Debate Transcript',
 'October 2, 2008 Debate Transcript',
 'October 7, 2008 Debate Transcript',
 'October 15, 2008 Debate Transcript',
 'October 13, 2004 Debate Transcript',
 'October 8, 2004 Debate Transcript',
 'October 5, 2004 Transcript',
 'September 30, 2004 Debate Transcript',
 'October 17, 2000 Debate Transcript',
 'October 11, 2000 Debate Transcript',
 'October 5, 2000 Debate Transcript',
 'October 3, 2000 Transcript',
 'October 16, 1996 Debate Transcript',
 'October 9, 1996 Debate Transcript',
 'October 6, 1996 Debate Transcript',
 'October 19, 1992 Debate Transcript',
 'October 15, 1992 First Half Debate Transcript',
 'October 

In [3]:
#removing first and last speeches manually (scraping isn't perfect)
urllist = urllist[1:-1]
titlelist = titlelist[1:-1]


In [4]:
#checks whether two strings are similar to one another, to deal with typos in the labeling of the speaker's name/ inconsistencies in naming
from difflib import SequenceMatcher
def isSimilar(x,y):
    simthreshhold = 0.5
    if x in y:
        return True
    if y in x:
        return True
    return SequenceMatcher(None, x, y).ratio() > simthreshhold

In [5]:
speechlist = []
for i in urllist:
    source1 = requests.get(i) 
    soup1 = bs.BeautifulSoup(source1.content, features='html.parser')

    speech = ''
    speechdf = pd.DataFrame()

    for p in soup1.find_all('p'):
        currspeaker = ''
        splittext = p.text.split(':',1)
        
        #if there's colon
        if len(splittext) == 2:
            name = splittext[0]
            speechtext =splittext[1]
            #if there's new speaker
            if (name.isupper()):
                currspeaker = name
                #if already in list of speakers
                if any(isSimilar(name,col) for col in speechdf.columns):
                    for cols in speechdf.columns:
                        if isSimilar(name, cols):
                        
                            currspeaker = cols
                    #set currspeaker to similar column
                    speechdf.loc[len(speechdf.index), currspeaker] = speechtext
                else: #if not in list of speakers
                    #create new column for new speaker
                    speechdf[currspeaker] = np.NaN
                    speechdf.loc[len(speechdf.index), currspeaker] = speechtext

   
    #continue paragraph
        elif len(currspeaker) > 1:
            speechdf.loc[len(speechdf.index), currspeaker] = p.text

    for col in speechdf.columns:
        if len(speechdf[col]) - speechdf[col].isna().sum() < 5:
            speechdf = speechdf.drop(col, axis = 1)     
    speechlist.append(speechdf)

    
    

In [6]:
#speechlist contains a list of dataframes, with each DF representing a presidential debate
len(speechlist)

44

In [16]:
#for speech in speechlist:

for speech in speechlist:
    sentdf = pd.DataFrame()
    for col in speech.columns:
        speechstring = ''
        for index,row in speech.iterrows():
            if isinstance(row[col],str):
                speechstring += row[col]
        
        blob = TextBlob(speechstring)
        #angry:-1 to happy: +1
        polarity = blob.sentiment.polarity
        sentdf[col] = np.NaN
        sentdf.loc[0, col] = polarity
    display(sentdf)
        
    

    
        

,WALLACE,CLINTON,TRUMP
0,0.137721,0.147258,0.051951


,RADDATZ,COOPER,QUESTION,CLINTON,TRUMP
0,0.08171,0.200084,0.329449,0.118943,0.179464


,QUIJANO,KAINE,PENCE
0,0.139518,0.140694,0.111796


,HOLT,CLINTON,TRUMP
0,0.099839,0.140567,0.150481


,[*]SCHIEFFER,ROMNEY,OBAMA
0,0.183612,0.163351,0.185358


...

,MR. REYNOLDS,MR. NEWMAN
0,0.131177,0.090412


,MR. NIXON,MR. EDWARDS
0,0.109811,0.129084


,MR. KENNEDY,MR. SHADEL,MR. DRUMMOND
0,0.080955,0.073138,0.152258


,MR. NIVEN
0,0.117754


,MR. SMITH,MR. WARREN
0,0.118435,0.100435


In [12]:
#example of how sentiment analysis works
string = 'hello happy person'
blob = TextBlob(string)
print(blob.sentiment.polarity)


0.8
